In [1]:
import pandas as pd
from pylab import rcParams
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.special import logsumexp
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import shap
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from numpy import mean
from numpy import where
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# Dataset Download

In [2]:
# Download and unzip Kaggle Data
# Ensure kaggle credentials are created and kaggle.json file  is present in the working directory
#!rm -r ~/.kaggle
#!mkdir ~/.kaggle
#!mv ./kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle competitions download -c caltech-cs155-2020
# unzip /content/caltech-cs155-2020.zip

In [3]:
# Get orderbook data
train_ob = pd.read_csv('C:/dev/Kaggle/train.csv')
print(train_ob.columns)

Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


In [4]:
train_ob

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592375,592375,4110.2,4110.3,1.0,1.0,2.0,1,4110.2,4110.0,4109.4,...,2,1,3,2,2,3,10,7,7,1
592376,592376,4109.4,4110.5,6.0,5.0,11.0,1,4109.2,4109.0,4108.6,...,2,2,6,5,3,9,7,7,5,0
592377,592377,4109.4,4110.5,NaN,NaN,0.0,0,4109.2,4109.0,4108.6,...,2,2,6,5,3,9,7,7,5,0
592378,592378,4109.4,4110.5,NaN,NaN,0.0,0,4109.2,4109.0,4108.6,...,2,2,6,5,3,9,7,7,5,0


In [5]:
test_ob = pd.read_csv('C:/dev/Kaggle/test.csv')
test_ob

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol
0,592380,5178.4,5178.3,1.0,11.0,12.0,-10,5178.2,5178.0,5177.8,...,3,16,3,1,1,1,4,1,5,2
1,592381,5133.0,5132.9,1.0,0.0,1.0,1,5132.8,5132.6,5132.4,...,1,2,2,1,10,4,8,1,1,1
2,592382,5177.4,5178.0,2.0,0.0,2.0,2,5177.2,5176.8,5176.6,...,8,18,1,1,6,7,4,1,2,5
3,592383,5093.6,5093.9,0.0,2.0,2.0,-1,5093.2,5093.0,5092.8,...,8,5,4,3,1,1,1,11,6,2
4,592384,5189.2,5189.2,NaN,NaN,0.0,0,5188.8,5188.6,5188.4,...,1,5,1,3,3,3,2,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,1,5,3,4,2,1,3,1,3,2
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,3,1,5,1,20,16,17,4,1,16
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,1,2,1,1,1,1,2,1,2,1
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,4,5,6,8,37,7,1,2,1,1


In [6]:
# Concat train and test to create additional features
df = pd.concat([train_ob,test_ob],sort=False)
print(df.shape)
print(df.columns)
df.tail()

df = df.dropna()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


# Calculation of Depth and Rise Ratio

In [7]:
# Calculate the total bid quantity and total ask quantity
df['total_bid_qty'] = df[['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']].sum(axis=1)
df['total_ask_qty'] = df[['ask1vol', 'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']].sum(axis=1)

# Calculate the bid depth ratio and ask depth ratio
df['bid_depth_ratio'] = df['total_bid_qty'] / df['total_ask_qty']
df['ask_depth_ratio'] = df['total_ask_qty'] / df['total_bid_qty']



df['mid'+'_rise_ratio'] = df['mid'].pct_change()

df['last_price'+'_rise_ratio'] = df['last_price'].pct_change()


df['mid_rise_ratio'] = df['mid_rise_ratio'].fillna(0)

df['last_price_rise_ratio'] = df['last_price_rise_ratio'].fillna(0)



print(df[['ask_depth_ratio','bid_depth_ratio','mid_rise_ratio','last_price_rise_ratio']])

        ask_depth_ratio  bid_depth_ratio  mid_rise_ratio  \
1              0.741935         1.347826        0.000000   
2              0.975610         1.025000        0.000234   
3              0.560976         1.782609       -0.000234   
4              0.911765         1.096774       -0.000078   
5              1.518519         0.658537        0.000286   
...                 ...              ...             ...   
592372         3.400000         0.294118        0.000049   
592373         2.666667         0.375000       -0.000243   
592374         1.642857         0.608696       -0.000073   
592375         2.900000         0.344828        0.000122   
592376         1.722222         0.580645        0.000049   

        last_price_rise_ratio  
1                    0.000000  
2                    0.000312  
3                   -0.000052  
4                   -0.000156  
5                    0.000104  
...                       ...  
592372               0.000341  
592373               0.

# Additional Feature Engineering

In [8]:
# Adding new features to data

# Create a dictionary mentioning bid and ask columns for creating additional features
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_avg"] = df[group_cols[group]].mean(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
    df[f"{group}_logsumexp"] = df[group_cols[group]].apply(logsumexp)


df["last_price_div__mid"] = df["last_price"].div(df["mid"])

# Synthetically adding a "date" column to the DataFrame and assigning timestamps to each row based on the "id" column.

df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


# Indentifying features with constant values


In [9]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")


# train = train.drop(["id","date","y"],axis=1)
# y = train["y"]

# sel = VarianceThreshold(threshold=0)
# sel.fit(train)

# print(
#     len([
#         x for x in train.columns
#         if x not in train.columns[sel.get_support()]
#     ]))

# [x for x in train.columns if x not in train.columns[sel.get_support()]]


X= train.drop(columns=['bid_cols_logsumexp',
 'bid_vol_cols_logsumexp',
 'ask_cols_logsumexp',
 'ask_vol_cols_logsumexp','date','id'])

X = X.drop(columns = ['bid1','bid2','bid3','bid4','bid5','ask5','ask1','ask2','ask3','ask4','bid_cols_max','bid_cols_min','ask_cols_max','ask_cols_min'])

y = X['y']

X = X.drop(columns = ['y'])

train shape 419920
test shape 0


# Imputing nan values with Regression based imputation


In [10]:

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Create an instance of IterativeImputer
imputer = IterativeImputer()

# Fit the imputer on your data (assuming X is your feature matrix)
imputer.fit(X)

# Impute the missing values
X = imputer.transform(X)

np.save('x_inputed',X)

X = np.load('x_inputed.npy')


# Calculate correlation between features


In [12]:
# Convert X to a pandas DataFrame
X_df = pd.DataFrame(X)

# Compute the correlation matrix
correlation_matrix = X_df.corr()

# Get the absolute correlation values and sort them in descending order
correlation_values = correlation_matrix.abs().unstack().sort_values(ascending=False)

# Filter out the correlation values for the same feature pairs and remove duplicates
correlation_values = correlation_values[correlation_values != 1].drop_duplicates()

# Print the most correlated feature pairs
most_correlated_features = []
for (feature1, feature2), correlation in correlation_values.items():
    most_correlated_features.append((feature1, feature2, correlation))
    if len(most_correlated_features) >= 30:  # Adjust the number of features as needed
        break

print("Most correlated features:")
for feature1, feature2, correlation in most_correlated_features:
    print(f"{feature1} - {feature2}: {correlation}")


Most correlated features:
16 - 25: 1.0000000000000056
31 - 17: 0.999999999999994
1 - 22: 0.9999984414964272
28 - 1: 0.9999984064689587
0 - 1: 0.9999967693400813
0 - 22: 0.9999954687220133
28 - 22: 0.9999954664231291
0 - 28: 0.9999954095520651
17 - 30: 0.9051302355455765
31 - 30: 0.9051302355455514
24 - 16: 0.8892082682275863
24 - 25: 0.8892082682275371
4 - 3: 0.8443287662859416
2 - 4: 0.7856203123871757
30 - 33: 0.7753564711793803
27 - 24: 0.755077193752881
17 - 19: 0.6980931492683544
19 - 31: 0.698093149268343
25 - 18: 0.6668969761358944
16 - 18: 0.6668969761358898
3 - 5: 0.645967382030227
30 - 19: 0.6424558624312792
19 - 18: 0.607857585919617
18 - 24: 0.6034615046966455
17 - 14: 0.5847664035954031
31 - 14: 0.5847664035953967
16 - 10: 0.5844098032149452
25 - 10: 0.5844098032149078
25 - 9: 0.5827316668761989
9 - 16: 0.5827316668761895


# Calculate Corretion Between Features and Target Variable

In [14]:
from scipy import stats

# Convert X to a pandas DataFrame
X_df = pd.DataFrame(X)

correlations = {}

for feature in X_df.columns:
    if X_df[feature].dtype == 'float64' or X_df[feature].dtype == 'int64':
        corr, _ = stats.pointbiserialr(X_df[feature], y)
        correlations[feature] = abs(corr)

# Sort the correlations in ascending order
sorted_correlations = sorted(correlations.items(), key=lambda x: x[1])

# Get the list of least correlated features and their correlation values
least_correlated_features = [feature for feature, _ in sorted_correlations]
correlation_values = [corr for _, corr in sorted_correlations]

# Print the list of least correlated features and their correlation values
for feature, corr in zip(least_correlated_features, correlation_values):
    print(f"{feature}: {corr}")


27: 0.0011717961011143544
10: 0.005065476665325711
9: 0.008009117564952376
21: 0.009474638210609954
14: 0.009494436344770943
8: 0.009835984663384563
13: 0.010302745083880748
15: 0.011839498099286954
33: 0.015345956410767954
1: 0.016010114842109135
22: 0.016059552631637344
28: 0.016174141364298025
0: 0.016385403518945955
5: 0.01705103878739411
12: 0.027195904803227085
7: 0.028544287868293584
24: 0.03044606563622956
30: 0.030857379513882014
2: 0.032598645580894224
16: 0.04423445227728988
25: 0.04423445227729187
3: 0.046322566243807956
4: 0.04889745366911359
26: 0.05056498166469449
23: 0.05067980318472081
17: 0.058851594742915306
31: 0.05885159474291698
19: 0.07534864861664961
29: 0.07798177243340722
6: 0.08078188166518727
32: 0.0849486909578949
18: 0.08640659848700738
20: 0.10435432877846976
11: 0.12126559242798864
34: 0.1479048383132955


# Splitting train into train and val sets


In [15]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(y_train.value_counts())

train_pool = Pool(data=X_train,label = y_train)


val_pool = Pool(data=X_val,label = y_val)


y
0.0    211249
1.0    124687
Name: count, dtype: int64


# Fixing class imbalance using smote oversampling


In [16]:

# transform the dataset
oversample = SMOTE()
undersample = RandomUnderSampler(sampling_strategy=0.8)
X_samp, y_samp = oversample.fit_resample(X_train, y_train)

print(X_samp.shape)
# X_samp, y_samp = undersample.fit_resample(X_samp, y_samp)

counter = Counter(y_samp)
print(counter)


(422498, 35)
Counter({1.0: 211249, 0.0: 211249})


# CatBoost Model training and hyperparameter tuning


In [17]:

from google.colab import output
output.enable_custom_widget_manager()


param_grid = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.1,0.05,0.01,0.001],
    'depth': [4, 6, 8,10],
    'l2_leaf_reg': [1, 3, 5,9],
    'bagging_temperature': [0.8, 1.0, 1.2],
    'border_count': [32, 64, 128],
    'eval_metric': ['Logloss'],
    'random_seed': [42]
}

model = CatBoostClassifier(task_type="GPU")

## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
# grid_search_result = model.randomized_search(grid,
#                                        train_pool,
#                                        n_iter=1,
#                                        plot=False,
#                                        refit = True, #  refit best model on all data
                                      # partition_random_seed=42)
# print(model.get_best_score())


random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=5, cv=3, random_state=42,verbose=2)
random_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy score
print("Best Parameters:", random_search.best_params_)
print("Best Accuracy Score:", random_search.best_score_)


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Fitting model on best params
best_params =  {'random_seed': 42, 'learning_rate': 0.1, 'l2_leaf_reg': 9, 'iterations': 1500, 'depth': 6, 'border_count': 64, 'bagging_temperature': 1.0}

train_pool_smote = Pool(data=X_train,label = y_train)

model = CatBoostClassifier(**best_params,eval_metric ='AUC',task_type="GPU")

model.fit(train_pool_smote, plot=True,silent=False)
print(model.get_best_score())

In [ ]:
# Metrics on validation set

import time

t1 = time.time()

print("Validation Set Length")
print(len(X_val))

y_pred = model.predict(X_val)

t2 = time.time()

tot = t2-t1

print(tot)

# Calculate evaluation metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

precision = precision_score(y_val, y_pred, average=None)
recall = recall_score(y_val, y_pred, average=None)
f1 = f1_score(y_val, y_pred, average=None)

# Calculate the confusion matrix
confusion_mat = confusion_matrix(y_val, y_pred)

# Print the results
class_names = np.unique(y)
for i, class_name in enumerate(class_names):
    print(f'Class: {class_name}')
    print(f'Precision: {precision[i]}')
    print(f'Recall: {recall[i]}')
    print(f'F1 Score: {f1[i]}')
    print('---')

print('Confusion Matrix:')
print(confusion_mat)


# Fit Randomforest Classifier Model


In [ ]:

clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val, predictions)
print("Test Set Accuracy: %.2f%%" % (accuracy * 100.0))

# Plot and extract the most important features for the model

plt.figure(num=None, figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
feat_importances = pd.Series(clf.feature_importances_, index= X.columns)
feat_importances.sort_values(ascending=False,inplace=True)

print("Most important Features")

imp_features = feat_importances.nlargest(20).index.tolist()

print(imp_features)

feat_importances.plot(kind='barh')

accuracy = accuracy_score(y_val, predictions)
precision = precision_score(y_val, predictions, average='weighted')
recall = recall_score(y_val, predictions, average='weighted')
f1 = f1_score(y_val, predictions, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

print('----------------------')


# Here it can be observed that

In [ ]:
# Getting final feature importances

feature_importances = model.get_feature_importance(train_pool)
feature_names = X.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    if score > 0.2:
        print('{0}: {1:.2f}'.format(name, score))

# Training XGBOOST model


In [ ]:

clf1 = XGBClassifier(
 verbosity=2,
 learning_rate =0.6,
 n_estimators=200,
 max_depth=12,
#  min_child_weight=1,
#  gamma=2,
#  subsample=0.8,
#  colsample_bytree=0.94,
 objective= 'binary:logistic',
#  nthread=4,
#  scale_pos_weight=2,
#  reg_alpha=0.5,
#  reg_lambda=0.5,
 seed=27)

clf1.fit(X_samp, y_samp)

y_pred = clf1.predict(X_val)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val, predictions)
print("Baseline Test Set Accuracy: %.2f%%" % (accuracy * 100.0))


# Validation of XGBOOST model


In [ ]:
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

print('----------------------')



precision = precision_score(y_val, y_pred, average=None)
recall = recall_score(y_val, y_pred, average=None)
f1 = f1_score(y_val, y_pred, average=None)

# Calculate the confusion matrix
confusion_mat = confusion_matrix(y_val, y_pred)

# Print the results
class_names = np.unique(y)
for i, class_name in enumerate(class_names):
    print(f'Class: {class_name}')
    print(f'Precision: {precision[i]}')
    print(f'Recall: {recall[i]}')
    print(f'F1 Score: {f1[i]}')
    print('---')

print('Confusion Matrix:')
print(confusion_mat)


#   XGBOOST hyperparameter tuning


In [ ]:

def objective(space):

    classifier = XGBClassifier(n_estimators = space['n_estimators'],
                            max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            gamma = space['gamma'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                            reg_alpha = space['reg_alpha'],
                            reg_lambda = space['reg_lambda'],
                            colsample_bytree = space['colsample_bytree'],
                            objective= 'binary:logistic',
                            scale_pos_weight = 5
                            )

    classifier.fit(X, y)

    # Applying k-Fold Cross Validation
    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,scoring='f1')
    CrossValMean = accuracies.mean()

    print("CrossValMean:", CrossValMean)

    return{'loss':1-CrossValMean, 'status': STATUS_OK }


# Possible values for various hyperparamters

space = {
    'max_depth' : hp.choice('max_depth', range(2, 10, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.6, 1, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 1500, 10)),
    'gamma' : hp.quniform('gamma', 2, 4, 0.1),
    'reg_alpha' : hp.quniform('reg_alpha', 0.3, 1, 0.1),
    'reg_lambda' : hp.quniform('reg_lambda', 0.3, 1, 0.1),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5,
            trials=trials)

print("Best: ", best)


# Extratrees Classifier

In [ ]:


from sklearn.ensemble import ExtraTreesClassifier

# Define the baseline hyperparameters
params = {
    'n_estimators': 300,
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'random_state': 42
}

# Create an instance of the ExtraTreesClassifier
model = ExtraTreesClassifier(**params)

# Fit the model to your training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

print('----------------------')


# AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score


model = AdaBoostClassifier(n_estimators=300, random_state=42)

# Fit the model to your training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

print('----------------------')

# SVM Classifier

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)



# Create a SVM classifier
clf = svm.SVC(kernel='rbf', C=1.0)  # You can choose the appropriate kernel type and C value

# Train the SVM classifier

X_train_scaled = X_train_scaled[:100000]
y_train = y_train[:100000]

clf.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = clf.predict(X_val_scaled)



# Evaluate the classifier performance
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

print('----------------------')

# LightGBM classifier

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)

# Set hyperparameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Train the LightGBM classifier
model = lgb.train(params, train_data, num_boost_round=100)

# Predict on the test set
y_pred = model.predict(X_val)
y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]

accuracy = accuracy_score(y_val, y_pred_binary)
precision = precision_score(y_val, y_pred_binary, average='weighted')
recall = recall_score(y_val, y_pred_binary, average='weighted')
f1 = f1_score(y_val, y_pred_binary, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

print('----------------------')

